## Sitting Posture Coach

https://www.sitting-posture.coach

The posture model is trained in this notebook.

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

Load the data using `keras.preprocessing.image_dataset_from_directory`. Training and validation data are in seperate directories, inside these directories are subdirectories `0` and `1` containing respectively bad and good sitting posture images. All images are 128x128 pixels and cropped around the persons head and shoulders. Take a look in `data.py` to see how these are generated and preprocessed.

In [ ]:
train_ds = keras.preprocessing.image_dataset_from_directory('data/train', batch_size=8, image_size=(128, 128))
val_ds = keras.preprocessing.image_dataset_from_directory('data/val', batch_size=8, image_size=(128, 128))

preprocessing_train = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.5),
    
])

preprocessing_val = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
])

train_ds = train_ds.map(lambda x, y: (preprocessing_train(x), y))
val_ds = val_ds.map(lambda x, y: (preprocessing_val(x), y))

x, y = next(iter(train_ds))
print(x.shape, y)

plt.figure(figsize=(10, 10))
for i in range(8):
    image = x[i]
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.axis("off")

In [ ]:
inputs = keras.Input(shape=(128, 128, 3))
x = layers.Conv2D(filters=16, kernel_size=(3, 3), activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=16, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=16, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=16, kernel_size=(3, 3), activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=3e-4), loss='binary_crossentropy')

model.summary()

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=35)

The trained model is saved directly in TensorFlow JavaScript (TFJS) format, in the right location in the frontend directory.

In [ ]:
tfjs.converters.save_keras_model(model, '../sitting-posture-coach/frontend/tfjs_model')

Results on the validation dataset.

In [ ]:
y_all, y_pred_all = [], []
for x, y in val_ds:
    y_pred = model.predict(x)
    y_all.append(y)
    y_pred_all.append(y_pred)
    print(f'label: {y}')
    print(f'prediction: {y_pred}')
y_all = np.concatenate(y_all).astype(np.float32)
y_pred_all = np.concatenate(y_pred_all)[:, 0]

sns.violinplot(x=y_all, y=y_pred_all)

Some code to generate saliency maps. What is the model looking for in the image to make a decision?

In [ ]:
with tf.GradientTape() as tape:
    tape.watch(x)
    y = model(x)
    # y_logit = tf.math.log(y / (1 - y))
    grad = tape.gradient(y, x)

In [ ]:
for i, (image, grad_image) in enumerate(zip(x, grad)):
    plt.figure(figsize=(10, 10))
    ax = plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.axis("off")
    ax = plt.subplot(1, 2, 2)
    plt.imshow(0.5 + grad_image * 50)
    plt.axis("off")